In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import edward as ed
import numpy as np
import tensorflow as tf
import six
import os

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from edward.models import Normal,Empirical,Bernoulli,Categorical
from tensorflow.contrib import slim
from tensorflow.contrib.keras.api.keras.layers import Dense

In [5]:
import matplotlib.pyplot as plt
import train_utils as util

In [6]:
%matplotlib inline

In [7]:
mtype = 'bpm'
iftype = 'VI'

In [8]:
N = 10000  # number of data points
M = 3 # number of features
H = 3 # number of classes
B = 64 # batch size during training

In [9]:
s_mean = 0.
s_std = 1.

d_mean = 0.
d_std = 3.

noise_std = .01

In [10]:
Y,X = util.build_toy_dataset(mtype,N,M-1,H,s_std,s_mean,d_std)

In [11]:
X.shape

(10000, 2)

In [12]:
lgr = LogisticRegression()
lgr.fit(X,Y)
print('Logistic regression accuracy: ',sum(lgr.predict(X)==Y)/N)

Logistic regression accuracy:  0.9736


In [13]:
X = np.hstack((X,np.ones((N,1))))

In [14]:
X = X.astype(np.float32)
#Y = Y.astype(np.float32)

x_ph = tf.placeholder(tf.float32, [B,M])
y_ph = tf.placeholder(tf.int32,[B])
y_ph_ohe = tf.placeholder(tf.float32,[B,H]) 

In [15]:
# model
w = Normal(tf.zeros([H-1,M]),tf.ones([H-1,M]))
    
if iftype == 'HMC':
    qw = Empirical(params=tf.Variable(tf.random_normal([B,H-1,M])))
else:
    qw = Normal(tf.Variable(tf.random_normal([H-1,M])), tf.nn.softplus(tf.Variable(tf.random_normal([H-1,M]))))

#y = Categorical(tf.nn.softmax(Normal(tf.matmul(x_ph,tf.transpose(w)), noise_std)))
y = Categorical(tf.nn.softmax(Normal(tf.concat([tf.matmul(x_ph,tf.transpose(w)),tf.zeros([B,1])],axis=1), noise_std)))


In [16]:
# inference
scaling = float(N) / B
nprint = 1000
niter = 20000

if iftype == 'EP':
    inference = ed.KLpq({w:qw},data={y:y_ph})
elif iftype == 'VI':
    inference = ed.KLqp({w:qw},data={y:y_ph})
elif iftype == 'HMC':
    inference = ed.HMC({w:qw},data={y:y_ph})
else:
    print('invalid inference type')
    
inference.initialize(n_iter=niter,n_print=nprint,scale={y:scaling})


In [17]:
sess = ed.get_session()
tf.global_variables_initializer().run()

In [18]:
ii = 0
sess = ed.get_session()
for t in range(niter):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
  
    info_dict = inference.update(feed_dict={x_ph:x_batch,y_ph:y_batch})
    inference.print_progress(info_dict)
    
    if t % nprint == 0:
        print('\n w mean:')
        print(sess.run(qw))

    1/20000 [  0%]                                ETA: 5256s | Loss: 11413.973

train_utils.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels!=None:
train_utils.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels == None:



 w mean:
[[-0.96580857 -1.27977908 -0.82098758]
 [ 0.92290175 -1.146209    0.17065722]]
 1000/20000 [  5%] █                              ETA: 14s | Loss: 5976.284 
 w mean:
[[ -3.19523335  -3.27656865  16.06370735]
 [ -1.14678884  -1.11376894  10.07752132]]
 2000/20000 [ 10%] ███                            ETA: 11s | Loss: 6264.770
 w mean:
[[ -3.58981442  -3.47344756  17.4291935 ]
 [ -1.25734782  -1.24564254  11.00165367]]
 3000/20000 [ 15%] ████                           ETA: 9s | Loss: 6282.323
 w mean:
[[ -3.48555875  -3.63012147  17.60212326]
 [ -1.27138913  -1.28093815  11.4271431 ]]
 4000/20000 [ 20%] ██████                         ETA: 8s | Loss: 6188.425
 w mean:
[[ -3.7020123   -3.43295479  17.93678284]
 [ -1.31483686  -1.28268027  11.39431953]]
 5000/20000 [ 25%] ███████                        ETA: 8s | Loss: 6022.579
 w mean:
[[ -3.61491346  -3.55017138  17.66453362]
 [ -1.31920087  -1.29545808  11.58558273]]
 6000/20000 [ 30%] █████████                      ETA: 7s | Los

In [19]:
ii = 0
acu = 0
y_test = tf.nn.softmax(tf.concat([tf.matmul(x_ph,tf.transpose(qw.loc)),tf.zeros([B,1])],axis=1))
for i in range(int(np.ceil(N/B))):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
    y_test_batch = sess.run(y_test,feed_dict={x_ph:x_batch,y_ph:y_batch})
    acu += sum(np.argmax(y_test_batch,axis=1)==y_batch)
print(acu*1./N)

0.9824
